In [58]:
import os
import pandas as pd
import librosa
import numpy as np

#### Filename identifiers 

1. Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
2. Vocal channel (01 = speech, 02 = song).
3. Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
4. Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
5. Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
6. Repetition (01 = 1st repetition, 02 = 2nd repetition).
7. Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).


In [33]:
def map_feeling_gender(feeling_list ,audio_list):
    for item in audio_list:
        if item[6:-16]=='02' and int(item[18:-4])%2==0:
            feeling_list.append('female_calm')
        elif item[6:-16]=='02' and int(item[18:-4])%2==1:
            feeling_list.append('male_calm')
        elif item[6:-16]=='03' and int(item[18:-4])%2==0:
            feeling_list.append('female_happy')
        elif item[6:-16]=='03' and int(item[18:-4])%2==1:
            feeling_list.append('male_happy')
        elif item[6:-16]=='04' and int(item[18:-4])%2==0:
            feeling_list.append('female_sad')
        elif item[6:-16]=='04' and int(item[18:-4])%2==1:
            feeling_list.append('male_sad')
        elif item[6:-16]=='05' and int(item[18:-4])%2==0:
            feeling_list.append('female_angry')
        elif item[6:-16]=='05' and int(item[18:-4])%2==1:
            feeling_list.append('male_angry')
        elif item[6:-16]=='06' and int(item[18:-4])%2==0:
            feeling_list.append('female_fearful')
        elif item[6:-16]=='06' and int(item[18:-4])%2==1:
            feeling_list.append('male_fearful')
#         elif item[:1]=='a':
#             feeling_list.append('male_angry')
#         elif item[:1]=='f':
#             feeling_list.append('male_fearful')
#         elif item[:1]=='h':
#             feeling_list.append('male_happy')
#         #elif item[:1]=='n':
#             #feeling_list.append('neutral')
#         elif item[:2]=='sa':
#             feeling_list.append('male_sad')

In [53]:
src_path = "../Dataset/Audio_Speech_Actors_01-24/Actor_0"
audio_list = []
feeling_list = []

for i in range(1,10):
    srcdir = src_path+str(i)
    temp = os.listdir(srcdir)
    for j in temp:
        audio_list.append(j)
        
src_path = "../Dataset/Audio_Speech_Actors_01-24/Actor_"
for i in range(10,25):
    srcdir = src_path+str(i)
    temp = os.listdir(srcdir)
    for j in temp:
        audio_list.append(j)
    

print(len(audio_list),"= 60 trials x 24 actors")
map_feeling_gender(feeling_list ,audio_list)

1440 = 60 trials x 24 actors


In [116]:
gender_list=[]
for i in range(len(feeling_list)):
    if "male" in feeling_list[i]:
        gender_list.append(0)
    else:
        gender_list.append(1)
        
labels = pd.DataFrame(feeling_list , columns=['label'])
labels_gender = pd.DataFrame(gender_list , columns=['label'])

### Feauture Extraction (mfcc)

In [77]:
src_path1= "../Dataset/Audio_Speech_Actors_01-24/Actor_0"
src_path2= "../Dataset/Audio_Speech_Actors_01-24/Actor_"

index=0
df=pd.DataFrame(columns=['feature'])
for index,y in enumerate(audio_list):
#     print(int(y[-6:-4]))
    if(int(y[-6:-4]) < 10):
        file = src_path1+str(int(y[-6:-4]))+"/"+y
    else:
        file = src_path2+str(int(y[-6:-4]))+"/"+y
    
    X, sample_rate = librosa.load(file,res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=13),axis=0)
    feature = mfccs
    df.loc[index] = [feature]
    index+=1
    
    

In [124]:
df_temp = pd.DataFrame(df['feature'].values.tolist())
df_new = pd.concat([df_temp,labels_gender], axis=1)

from sklearn.utils import shuffle
df_new = shuffle(df_new)
df_new[:10]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,label
793,-65.339569,-65.339569,-65.339569,-65.339569,-65.339569,-65.339569,-65.339569,-65.339569,-65.339569,-65.339569,...,-63.259981,-62.485473,-60.542394,-60.430376,-61.678093,-62.865829,-59.956404,-58.453123,-62.739386,0.0
634,-51.725560,-54.124322,-55.544334,-54.201736,-55.323090,-54.073407,-52.928944,-52.014225,-50.625279,-49.931809,...,-56.343736,-58.966647,-60.412907,-57.063297,-56.789003,-58.956166,-66.689059,-68.334259,-64.642337,0.0
736,-57.756826,-56.945218,-55.402335,-54.326853,-54.621980,-55.201250,-57.895613,-57.895613,-56.603973,-56.500033,...,-29.067356,-28.063373,-26.223268,-27.358451,-30.899799,-33.100692,-33.505974,-35.205846,-35.774270,0.0
674,-56.949561,-57.069997,-56.832876,-56.703924,-57.132519,-57.132519,-57.124029,-55.651101,-54.768143,-54.359553,...,-54.195580,-53.350789,-53.221771,-53.519392,-54.016182,-54.139134,-55.325821,-55.854896,-57.031073,0.0
591,-46.337760,-46.472007,-47.163061,-47.608013,-46.827917,-47.003932,-48.554300,-48.514036,-48.063489,-47.983162,...,-48.131800,-47.385767,-46.406479,-47.405863,-46.007680,-44.447033,-43.594649,-44.923551,-48.167015,0.0
816,-55.458033,-55.458033,-55.458033,-55.458033,-55.458033,-55.458033,-55.458033,-55.458033,-55.458033,-55.458033,...,-53.233205,-53.706885,-53.124855,-51.982507,-54.533270,-55.047042,-54.817246,-50.836242,-50.490346,0.0
279,-53.812760,-54.138708,-55.172711,-57.848372,-56.782412,-56.270149,-57.019021,-59.633390,-60.433832,-59.938604,...,-34.716900,-34.928820,-36.931785,-39.506352,-41.009955,-41.554357,-41.179449,-43.274930,-47.514118,0.0
939,-53.868375,-52.661949,-52.405631,-53.798625,-55.014937,-54.927314,-54.938003,-55.730871,-54.232794,-55.141933,...,-34.019282,-34.204850,-33.312031,-27.541980,-22.581987,-22.915310,-28.673290,-27.136780,-23.094558,0.0
342,-42.751073,-42.751073,-42.719816,-42.308264,-41.886064,-41.338156,-41.120344,-40.376949,-40.439036,-40.034355,...,-34.810295,-37.023740,-37.102847,-35.540489,-35.563585,-37.408276,-38.206487,-37.845648,-38.973762,0.0
1111,-55.102562,-56.048294,-58.384500,-61.521559,-64.581595,-64.264367,-63.681615,-61.965197,-63.487459,-64.640674,...,-51.864803,-52.473707,-51.908845,-54.379388,-53.660037,-53.956100,-52.871700,-51.789471,-52.396382,NaN


In [125]:
## making NaN as 0
df_new = df_new.fillna(0)

### Naive Bayes Classifier ( Binary Classification )

### Train-Test Split 

In [126]:
random_split = np.random.rand(len(df_new)) < 0.7
train = df_new[random_split]
test = df_new[~random_split]

trainfeatures = train.iloc[:, :-1]
trainlabel = train.iloc[:, -1:]
testfeatures = test.iloc[:, :-1]
testlabel = test.iloc[:, -1:]

In [127]:
X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

In [128]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [129]:
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0
